# Image gathering
The robot takes pictures in 5 degree increments to familiarize itself with its environment.

In [1]:
import cozmo
from cozmo.util import degrees
import cv2
import os
from PIL import Image as im 
import numpy as np

Set up the folder in which to store the images.

In [2]:
# File path configuration
parent_dir = r"C:\Users\Dressler\Pictures"
path = os.path.join(parent_dir, "raw")

# Only needs to be run once
try:
    os.mkdir(path)
except FileExistsError:
    print("Directory already exists")

Directory already exists


Take the pictures, storing them in the specified directory

In [3]:
def take_pictures(robot: cozmo.robot.Robot):
  # Initial setup, generate repeatable head angle and lift height
  robot.say_text("Taking pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True

  deg_step = 5
  
  # Saves picture of what Cozmo sees every 10 degrees.
  for d in range(0, 360, deg_step):
    fileName = "image" + str(d) + ".jpeg"
 
    robot.turn_in_place(degrees(deg_step)).wait_for_completed()
    
    latest_image = robot.world.latest_image
    # annotation seems to be important, we have had issues saving images if we don't annotate
    #annotations_enabled=None
    #annotated = latest_image.annotate_image()

    if latest_image is not None:
      print("image = %s" % latest_image)
      #converted = annotated.convert()
      file_path = os.path.join(path, fileName)
      latest_image.raw_image.save(file_path, "JPEG", resolution=10)

Method to downscale the images for faster processing later on.

In [4]:
def resize_saved(width, height):
    deg_step = 5
    for d in range(0, 360, deg_step):
        fileName = "image" + str(d) + ".jpeg"
        path = r"C:\Users\Dressler\Pictures\raw"
        source_path = os.path.join(path, fileName)
        
        image = cv2.imread(source_path)
        res = cv2.resize(image, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
    
        rImg = im.fromarray(res)
        #TODO: mkdir
        rPath = r"C:\Users\Dressler\Pictures\resized0"
        resized_path = os.path.join(rPath, fileName)
        rImg.save(resized_path, "JPEG", resolution=10)

In [5]:
cozmo.run_program(take_pictures)
resize_saved(270, 180)

c:\Users\Dressler\AppData\Local\Programs\Python\Python39\lib\site-packages\cozmo\event.py:488: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  return await asyncio.wait_for(f, timeout, loop=self._loop)
2024-04-30 22:43:26,602 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2024-04-30 22:43:26,607 cozmo.general INFO     Found robot id=1
2024-04-30 22:43:26,623 cozmo.general INFO     Connected to Android device serial=1A071FDF600FFW
2024-04-30 22:43:26,900 cozmo.general INFO     Robot id=1 serial=4322be95 initialized OK


image = <cozmo.world.CameraImage object at 0x000001DE55025880>
image = <cozmo.world.CameraImage object at 0x000001DE55025CA0>
image = <cozmo.world.CameraImage object at 0x000001DE55025C40>
image = <cozmo.world.CameraImage object at 0x000001DE55049400>
image = <cozmo.world.CameraImage object at 0x000001DE44A8FF40>
image = <cozmo.world.CameraImage object at 0x000001DE5505D310>
image = <cozmo.world.CameraImage object at 0x000001DE55049DC0>
image = <cozmo.world.CameraImage object at 0x000001DE5501EFA0>
image = <cozmo.world.CameraImage object at 0x000001DE551B5820>
image = <cozmo.world.CameraImage object at 0x000001DE44A8FF40>
image = <cozmo.world.CameraImage object at 0x000001DE54FD5A00>
image = <cozmo.world.CameraImage object at 0x000001DE5509B6D0>
image = <cozmo.world.CameraImage object at 0x000001DE550C5520>
image = <cozmo.world.CameraImage object at 0x000001DE550255E0>
image = <cozmo.world.CameraImage object at 0x000001DE551B5FA0>
image = <cozmo.world.CameraImage object at 0x000001DE55

2024-04-30 22:44:23,683 cozmo.general INFO     Shutting down connection
2024-04-30 22:44:23,689 cozmo.general INFO     Android serial=1A071FDF600FFW disconnected.


image = <cozmo.world.CameraImage object at 0x000001DE44A8FF40>
